In [1]:
import warnings
import sys
import pathlib
import torch
import gpytorch
from IPython.display import HTML


warnings.filterwarnings('ignore')
sys.path.insert(0, str(pathlib.Path.cwd()/"src"))
%load_ext autoreload
%autoreload 2

import loaders.hrrr
import fields.continuous

In [2]:
import loaders.hrrr
import fields.discrete
import fields.continuous
import models.gp
import models.neural_flow

In [46]:
hrrr = loaders.hrrr.Data(date="2024-09-26 17:00", hours=4, level= 700)

✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F00 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F01 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F02 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F03 ┊ GRIB2 @ aws ┊ IDX @ aws
✅ Found ┊ model=hrrr ┊ product=sfc ┊ 2024-Sep-26 17:00 UTC F04 ┊ GRIB2 @ aws ┊ IDX @ aws


In [47]:
dvf = hrrr.vector_field(extent=(-88,-81,24.5,29))
dsf = hrrr.scalar_field(extent=(-88,-81,24.5,29))

In [48]:
HTML(dsf.plot().to_html5_video())

In [106]:
cvf0 = fields.continuous.VectorField()
cvf0.train_mse_vector(dvf, batch_size=1000, epochs = 50)


Iter 1/750 — loss: 2.2086
Iter 2/750 — loss: 2.2413
Iter 3/750 — loss: 2.2420
Iter 4/750 — loss: 2.2056
Iter 5/750 — loss: 2.2518
Iter 6/750 — loss: 2.2442
Iter 7/750 — loss: 2.2110
Iter 8/750 — loss: 2.2134
Iter 9/750 — loss: 2.1144
Iter 10/750 — loss: 2.0941
Iter 11/750 — loss: 2.1600
Iter 12/750 — loss: 2.0937
Iter 13/750 — loss: 2.0910
Iter 14/750 — loss: 2.1668
Iter 15/750 — loss: 2.0648
Iter 16/750 — loss: 2.1882
Iter 17/750 — loss: 2.0503
Iter 18/750 — loss: 1.9915
Iter 19/750 — loss: 2.0346
Iter 20/750 — loss: 1.9303
Iter 21/750 — loss: 1.9905
Iter 22/750 — loss: 1.9141
Iter 23/750 — loss: 1.9049
Iter 24/750 — loss: 1.9049
Iter 25/750 — loss: 1.8492
Iter 26/750 — loss: 1.7986
Iter 27/750 — loss: 1.7443
Iter 28/750 — loss: 1.7351
Iter 29/750 — loss: 1.6809
Iter 30/750 — loss: 1.6749
Iter 31/750 — loss: 1.5843
Iter 32/750 — loss: 1.5405
Iter 33/750 — loss: 1.5359
Iter 34/750 — loss: 1.4927
Iter 35/750 — loss: 1.4621
Iter 36/750 — loss: 1.4128
Iter 37/750 — loss: 1.3534
Iter 38/75

In [107]:
HTML(cvf0.plot(dvf.coord_field).to_html5_video())

In [ ]:
flow = cvf0.flow
Z = dsf.simulate(flow)
dsf0 = fields.discrete.ScalarField(dsf.coord_field, Z)

In [161]:
HTML(dsf.plot().to_html5_video())

In [162]:
cvf = fields.continuous.VectorField()
cvf.train_mle(dsf, batch_size=5*2907//2, epochs = 20)

Epoch 1/20 — loss: 0.8737 — noise_var: 0.5982 — lengthscale: [1.8468337  0.4698124  0.47015297]


Epoch 2/20 — loss: 0.8032 — noise_var: 0.5131 — lengthscale: [1.716931   0.53905654 0.5471841 ]


Epoch 3/20 — loss: 0.7316 — noise_var: 0.4374 — lengthscale: [1.6048607  0.6048945  0.62900364]


Epoch 4/20 — loss: 0.6715 — noise_var: 0.3709 — lengthscale: [1.4893974  0.66055155 0.71146965]


Epoch 5/20 — loss: 0.6112 — noise_var: 0.3132 — lengthscale: [1.3803298  0.68675727 0.78935045]


...

Epoch 16/20 — loss: 0.2668 — noise_var: 0.0608 — lengthscale: [0.5412344  0.36244562 0.5289078 ]


Epoch 17/20 — loss: 0.2683 — noise_var: 0.0565 — lengthscale: [0.5021627  0.3510821  0.48938394]


Epoch 18/20 — loss: 0.2675 — noise_var: 0.0533 — lengthscale: [0.486349   0.33916047 0.470636  ]


Epoch 19/20 — loss: 0.2571 — noise_var: 0.0511 — lengthscale: [0.48691386 0.32342905 0.44771558]


Epoch 20/20 — loss: 0.2609 — noise_var: 0.0496 — lengthscale: [0.47997242 0.30702475 0.42323333]


In [163]:
HTML(cvf.plot(dvf.coord_field).to_html5_video())

In [157]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [114]:
dsf.coord_field.TXY.shape[0]/5

2907.0

In [115]:
cvf.velocity(dsf.coord_field.TXY_scaled)

tensor([[ 0.0396, -0.6767],
        [ 0.0386, -0.6403],
        [ 0.0366, -0.6049],
        ...,
        [ 0.6646,  1.6766],
        [ 0.6380,  1.6797],
        [ 0.6114,  1.6828]], grad_fn=<AddmmBackward0>)

In [129]:
torch.norm(cvf0.velocity(dsf.coord_field.TXY_scaled) - cvf.velocity(dsf.coord_field.TXY_scaled), dim = -1).mean()

tensor(0.2453, grad_fn=<MeanBackward0>)

In [99]:
dsf.coord_field.TXY_scaled.device

device(type='cpu')

In [ ]:
torch.func.jacrev